# xDB Training

A notebook exploring the xDB dataset tier3 part for Building Segmentation Model and Damaged Classification Model

In [1]:
# In a Jupyter notebook or IPython environment, run this in the first cell
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [4]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
# Define the transformation pipeline
def get_train_augmentation_pipeline(image_size=(256, 256)):
    transform = A.Compose([
        # Resize images and masks
        A.Resize(image_size[0], image_size[1], p=1.0),  # Ensure both image and mask are resized
        # Random horizontal flip
        A.HorizontalFlip(p=0.5),
        # Random vertical flip
        A.VerticalFlip(p=0.5),
        # Random rotation
        A.RandomRotate90(p=0.5),
        # Random brightness and contrast adjustments
        A.RandomBrightnessContrast(p=0.2),
        # Random contrast adjustment
        A.RandomGamma(p=0.2),
        # Random scale and aspect ratio change
        A.RandomSizedCrop(min_max_height=(image_size[0], image_size[1]), height=image_size[0], width=image_size[1], p=0.5),
        # Random blur
        A.GaussianBlur(p=0.2),
        # Random crop (if you want to focus on specific regions)
        A.CenterCrop(height=image_size[0], width=image_size[1], p=1.0),
        # Convert to tensor (works for both image and mask)
        ToTensorV2()
    ])
    return transform

def get_val_augmentation_pipeline(image_size=(256, 256)):
    transform = A.Compose([
        # Resize images and masks
        A.Resize(image_size[0], image_size[1], p=1.0),  # Ensure both image and mask are resized
        ToTensorV2()
    ])
    return transform

In [5]:
from datasets import xDB_Damaged_Building

origin_dir = "../data/xDB/tier3"

data_train  = xDB_Damaged_Building(
    origin_dir = origin_dir,
    mode="building",
    time="pre",
    transform=get_train_augmentation_pipeline(image_size=(512,512)),
    type="train",
    val_ratio=0.2, 
    test_ratio=0.1,
)

data_val  = xDB_Damaged_Building(
    origin_dir = origin_dir,
    mode="building",
    time="pre",
    transform=get_val_augmentation_pipeline(image_size=(512,512)),
    type="val",
    val_ratio=0.2, 
    test_ratio=0.1,
)

Loaded 4460 train labels.
Loaded 1273 val labels.


In [6]:
from models import ResNet_UNET 

model =  ResNet_UNET(
        in_channels=3,
        out_channels=2,
        backbone_name="resnet18",
        pretrained=True,
        freeze_backbone=False,
    ).to("cuda")

In [7]:
from torch.utils.data import DataLoader
train_dl = DataLoader(data_train, batch_size=8, shuffle=True,num_workers=4, pin_memory=True)
val_dl = DataLoader(data_val, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from training import train 
from losses import SoftCrossEntropyLoss
from metrics import accuracy, f1_score, iou_score


optimizer = optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
params_opt = {}
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10,gamma=0.1)
params_sc = {}
criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.3, 0.7]).to("cuda"))
# Define Metrics 
metrics = [accuracy, f1_score, iou_score]

# Early Stopping 
early_stopping_params = {"patience":5, "trigger_times": 0}

train(
    model,
    train_dl=train_dl,
    valid_dl=val_dl,
    optimizer=optimizer,
    scheduler=scheduler,
    params_opt=params_opt,
    params_sc=params_sc,
    metrics=metrics,
    nb_epochs=15,
    loss_fn=criterion,
    experiment_name="xDB_UNet",
    log_dir="../runs",
    model_dir="../models",
    early_stopping_params = early_stopping_params,
    image_key="image"
)

Training will be done on  cuda
Metric computations is based on multiclass mode
Mixed Precision Training : False
Experiment logs are recoded at ../runs\xDB_UNet_20241121-233922
Epoch 1
----------


Epoch 1: 100%|██████████| 558/558 [02:07<00:00,  4.39batch/s]


Epoch 1 Training completed. Loss: 0.3653

Metrics (Training Phase):

+-----------+--------+
|   Metric  | Value  |
+-----------+--------+
|  accuracy | 0.0000 |
|  f1_score | 0.0000 |
| iou_score | 0.0000 |
+-----------+--------+


Validation Epoch 1: 100%|██████████| 160/160 [00:36<00:00,  4.38batch/s]


Epoch 1 Validation completed. Loss: 0.3314

Metrics (Validation Phase):

+-----------+--------+
|   Metric  | Value  |
+-----------+--------+
|  accuracy | 0.0000 |
|  f1_score | 0.0000 |
| iou_score | 0.0000 |
+-----------+--------+
Training Loss : 0.365276790894735 / Validation : 0.3314170478042054
Epoch 2
----------


...

Epoch 3: 100%|██████████| 558/558 [02:04<00:00,  4.47batch/s]


Epoch 3 Training completed. Loss: 0.2543

Metrics (Training Phase):

+-----------+--------+
|   Metric  | Value  |
+-----------+--------+
|  accuracy | 0.0000 |
|  f1_score | 0.0000 |
| iou_score | 0.0000 |
+-----------+--------+


Validation Epoch 3: 100%|██████████| 160/160 [00:36<00:00,  4.37batch/s]


Epoch 3 Validation completed. Loss: 0.2588

Metrics (Validation Phase):

+-----------+--------+
|   Metric  | Value  |
+-----------+--------+
|  accuracy | 0.0000 |
|  f1_score | 0.0000 |
| iou_score | 0.0000 |
+-----------+--------+
Training Loss : 0.2542824187620873 / Validation : 0.2587649804559732


KeyboardInterrupt: 